In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/users/alfon/Desktop/Master/TFM/CSV/cars_final_colour.csv')
df.head()

,Brand,Model,Version,Category,Year,Kms,Hp,Gear_type,Fuel_type,Fuel_cons,Doors,Colour,ZIP,Province,Price_EUR
0,Fiat,Tipo,1.4 Easy,Small,2017,24000,95,Manual,Petrol,6.5,5,Beige,43001,Tarragona,10900
1,SEAT,Leon,2.0TDI CR S&S FR Fast Edition DSG7 150,Small,2020,14122,150,Automatic,Diesel,4.3,5,Blue,43006,Tarragona,24200
2,Audi,A3,Sportback 1.6TDI Design Edition 81kW,Small,2017,65290,110,Manual,Diesel,3.9,5,Blue,43006,Tarragona,19200
3,Honda,Civic,1.0 VTEC Turbo Elegance Navi,Small,2018,54189,129,Manual,Petrol,5.1,5,Blue,43204,Tarragona,18900
4,Volkswagen,Beetle,1.4 TSI Sport 160,Small,2013,105300,159,Manual,Petrol,6.6,5,Brown,43001,Tarragona,19900


Removing unnecesary columns for the study

In [8]:
df = df.drop(columns = ["Version","ZIP"], axis = 1)


## Features

In [9]:
df.dtypes

Brand         object
Model         object
Category      object
Year           int64
Kms            int64
Hp             int64
Gear_type     object
Fuel_type     object
Fuel_cons    float64
Doors          int64
Colour        object
Province      object
Price_EUR      int64
dtype: object

There are two kind of features in this dataset: Numerical and Categorical Features

In [10]:
#Numerical Features
df.select_dtypes(exclude=["object"]).columns.value_counts()

Year         1
Kms          1
Hp           1
Fuel_cons    1
Doors        1
Price_EUR    1
dtype: int64

In [11]:
#Categorical Features
df.select_dtypes(exclude=["number"]).columns.value_counts()

Brand        1
Model        1
Category     1
Gear_type    1
Fuel_type    1
Colour       1
Province     1
dtype: int64

To be able to analyze the correlation between all features of the dataset, including numerical and categorical features we need to transform categorical into numerical

## Transformating Categorical features to Numerical features 


By using a one hot encoder we are going to include some of the categorical features to the data frame columns, tranforming them to dummy columns coding them 0 and 1

###  Option 1: One hot encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe_gear = OneHotEncoder().fit_transform(df[["Gear_type"]]).toarray()

In [ ]:
ohe_gear

In [ ]:
df = df.join(pd.DataFrame(ohe_gear))

In [ ]:
df

### Optoin 2 : Dummies

Transforming categorical columns to Category for those features that make sense to conver to dummies columns 

In [13]:
for col in ['Category','Gear_type', 'Fuel_type']:
    df[col] = df[col].astype('category')
print(df.dtypes)

Brand          object
Model          object
Category     category
Year            int64
Kms             int64
Hp              int64
Gear_type    category
Fuel_type    category
Fuel_cons     float64
Doors           int64
Colour         object
Province       object
Price_EUR       int64
dtype: object


In [14]:
df = pd.get_dummies(data=df,columns=['Category','Gear_type','Fuel_type'])

In [15]:
df.head()

,Brand,Model,Year,Kms,Hp,Fuel_cons,Doors,Colour,Province,Price_EUR,...,Category_Suv,Category_Van,Category_Wagon,Gear_type_Automatic,Gear_type_Manual,Fuel_type_Compressed Natural Gas \(CNG\),Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Petrol
0,Fiat,Tipo,2017,24000,95,6.5,5,Beige,Tarragona,10900,...,0,0,0,0,1,0,0,0,0,1
1,SEAT,Leon,2020,14122,150,4.3,5,Blue,Tarragona,24200,...,0,0,0,1,0,0,1,0,0,0
2,Audi,A3,2017,65290,110,3.9,5,Blue,Tarragona,19200,...,0,0,0,0,1,0,1,0,0,0
3,Honda,Civic,2018,54189,129,5.1,5,Blue,Tarragona,18900,...,0,0,0,0,1,0,0,0,0,1
4,Volkswagen,Beetle,2013,105300,159,6.6,5,Brown,Tarragona,19900,...,0,0,0,0,1,0,0,0,0,1


### Option 3 : Encoder for high cardinality features

In [ ]:
conda install -c conda-forge category_encoders

In [16]:
from category_encoders import TargetEncoder

In [17]:
df.select_dtypes(exclude=["number"])

,Brand,Model,Colour,Province
0,Fiat,Tipo,Beige,Tarragona
1,SEAT,Leon,Blue,Tarragona
2,Audi,A3,Blue,Tarragona
3,Honda,Civic,Blue,Tarragona
4,Volkswagen,Beetle,Brown,Tarragona
...,...,...,...,...
20006,Kia,XCeed,Orange,Teruel
20007,BMW,M4,Yellow,Segovia
20008,BMW,M4,Yellow,Segovia
20009,Mercedes-Benz,E 220,Silver,Segovia


In [19]:
encoder = TargetEncoder()

In [22]:
cols_to_encode = ['Brand', 'Model', 'Colour', 'Province']
cols_encoded = list(map(lambda c: c + '_encoded', cols_to_encode))

df[cols_encoded] = encoder.fit_transform(df[cols_to_encode], df.Price_EUR)

In [24]:
df.head()

,Brand,Model,Year,Kms,Hp,Fuel_cons,Doors,Colour,Province,Price_EUR,...,Gear_type_Manual,Fuel_type_Compressed Natural Gas \(CNG\),Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Petrol,Brand_encoded,Model_encoded,Colour_encoded,Province_encoded
0,Fiat,Tipo,2017,24000,95,6.5,5,Beige,Tarragona,10900,...,1,0,0,0,0,1,11624.755365,11660.903614,15819.395186,36813.324324
1,SEAT,Leon,2020,14122,150,4.3,5,Blue,Tarragona,24200,...,0,0,1,0,0,0,15357.711086,16418.168342,25942.629843,36813.324324
2,Audi,A3,2017,65290,110,3.9,5,Blue,Tarragona,19200,...,1,0,1,0,0,0,26918.515604,18714.449704,25942.629843,36813.324324
3,Honda,Civic,2018,54189,129,5.1,5,Blue,Tarragona,18900,...,1,0,0,0,0,1,12803.411765,13193.843084,25942.629843,36813.324324
4,Volkswagen,Beetle,2013,105300,159,6.6,5,Brown,Tarragona,19900,...,1,0,0,0,0,1,18031.517024,15010.734375,22221.022810,36813.324324


In [25]:
df.drop(['Brand', 'Model', 'Colour', 'Province'], axis = 1, inplace = True)


,Year,Kms,Hp,Fuel_cons,Doors,Price_EUR,Category_Convertible,Category_Coupe,Category_Minivan,Category_Sedan,...,Gear_type_Manual,Fuel_type_Compressed Natural Gas \(CNG\),Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Petrol,Brand_encoded,Model_encoded,Colour_encoded,Province_encoded
0,2017,24000,95,6.5,5,10900,0,0,0,0,...,1,0,0,0,0,1,11624.755365,11660.903614,15819.395186,36813.324324
1,2020,14122,150,4.3,5,24200,0,0,0,0,...,0,0,1,0,0,0,15357.711086,16418.168342,25942.629843,36813.324324
2,2017,65290,110,3.9,5,19200,0,0,0,0,...,1,0,1,0,0,0,26918.515604,18714.449704,25942.629843,36813.324324
3,2018,54189,129,5.1,5,18900,0,0,0,0,...,1,0,0,0,0,1,12803.411765,13193.843084,25942.629843,36813.324324
4,2013,105300,159,6.6,5,19900,0,0,0,0,...,1,0,0,0,0,1,18031.517024,15010.734375,22221.022810,36813.324324


Lets re organize the data frame, moving the Target Price to the last column

In [35]:
df_features = df.loc[:,df.columns != "Price_EUR"]
df_features.head()

,Year,Kms,Hp,Fuel_cons,Doors,Category_Convertible,Category_Coupe,Category_Minivan,Category_Sedan,Category_Small,...,Gear_type_Manual,Fuel_type_Compressed Natural Gas \(CNG\),Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Petrol,Brand_encoded,Model_encoded,Colour_encoded,Province_encoded
0,2017,24000,95,6.5,5,0,0,0,0,1,...,1,0,0,0,0,1,11624.755365,11660.903614,15819.395186,36813.324324
1,2020,14122,150,4.3,5,0,0,0,0,1,...,0,0,1,0,0,0,15357.711086,16418.168342,25942.629843,36813.324324
2,2017,65290,110,3.9,5,0,0,0,0,1,...,1,0,1,0,0,0,26918.515604,18714.449704,25942.629843,36813.324324
3,2018,54189,129,5.1,5,0,0,0,0,1,...,1,0,0,0,0,1,12803.411765,13193.843084,25942.629843,36813.324324
4,2013,105300,159,6.6,5,0,0,0,0,1,...,1,0,0,0,0,1,18031.517024,15010.734375,22221.022810,36813.324324


In [39]:
df_target = df[["Price_EUR"]]
df_target.head()

,Price_EUR
0,10900
1,24200
2,19200
3,18900
4,19900


In [41]:
df = df_features.join(df_target)
df.head()

,Year,Kms,Hp,Fuel_cons,Doors,Category_Convertible,Category_Coupe,Category_Minivan,Category_Sedan,Category_Small,...,Fuel_type_Compressed Natural Gas \(CNG\),Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Petrol,Brand_encoded,Model_encoded,Colour_encoded,Province_encoded,Price_EUR
0,2017,24000,95,6.5,5,0,0,0,0,1,...,0,0,0,0,1,11624.755365,11660.903614,15819.395186,36813.324324,10900
1,2020,14122,150,4.3,5,0,0,0,0,1,...,0,1,0,0,0,15357.711086,16418.168342,25942.629843,36813.324324,24200
2,2017,65290,110,3.9,5,0,0,0,0,1,...,0,1,0,0,0,26918.515604,18714.449704,25942.629843,36813.324324,19200
3,2018,54189,129,5.1,5,0,0,0,0,1,...,0,0,0,0,1,12803.411765,13193.843084,25942.629843,36813.324324,18900
4,2013,105300,159,6.6,5,0,0,0,0,1,...,0,0,0,0,1,18031.517024,15010.734375,22221.022810,36813.324324,19900


In [42]:

df.dtypes

Year                                          int64
Kms                                           int64
Hp                                            int64
Fuel_cons                                   float64
Doors                                         int64
Category_Convertible                          uint8
Category_Coupe                                uint8
Category_Minivan                              uint8
Category_Sedan                                uint8
Category_Small                                uint8
Category_Suv                                  uint8
Category_Van                                  uint8
Category_Wagon                                uint8
Gear_type_Automatic                           uint8
Gear_type_Manual                              uint8
Fuel_type_Compressed Natural Gas \(CNG\)      uint8
Fuel_type_Diesel                              uint8
Fuel_type_Electric                            uint8
Fuel_type_Hybrid                              uint8
Fuel_type_Pe

All the features from the DataFrame have been transformed to numerical

In [45]:
df.to_csv('/users/alfon/Desktop/Master/TFM/CSV/cars_features.csv',header=True, index=False)